In [ ]:
print("문제 2-4 : 여행 계획 분석기")

In [1]:
from dotenv import load_dotenv
import os
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

response_schemas = [
    ResponseSchema(name="destination", description="여행지 이름"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
    ResponseSchema(name="budget", description="여행 예산 (예: 30만원)"),
    ResponseSchema(name="rating", description="여행 추천도, 1~5 사이의 숫자"),
    ResponseSchema(name="activities", description="여행 중 주요 활동 리스트")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 후기나 계획을 분석하여 구조화된 데이터를 생성하는 전문가다."),
    ("human", "다음 텍스트에서 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 추출하라.\n"
              "출력은 반드시 다음 형식을 따라야 한다:\n{format_instructions}\n\n"
              "여행 텍스트: {travel_text}")
])

chain = prompt | llm | output_parser

travel_text = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 "
    "해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. "
    "정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

result = chain.invoke({
    "travel_text": travel_text,
    "format_instructions": output_parser.get_format_instructions()
})

print(result)

{'destination': '부산', 'duration': '2박3일', 'budget': '30만원', 'rating': '4', 'activities': '해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경'}
